In [1]:
from mistralai import Mistral
import yaml
import json
import requests 
from tqdm.auto import notebook_tqdm
import minsearch
# from elasticsearch import Elasticsearch

/home/codespace/.python/current/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
api_key = "ollama"
model = "mistral"

In [4]:
client = Mistral(server_url="http://127.0.0.1:11434", api_key=api_key)

In [5]:
chat_response = client.chat.complete(
    model=model,
    messages=[{"role":"user", "content":"What is the best French cheese?"}]
)

SDKError: API error occurred: Status 500
{"error":{"message":"model requires more system memory (5.5 GiB) than is available (5.2 GiB)","type":"api_error","param":null,"code":null}}


Aqui puedo modificar la URL a la que apunta el codigo, en lugar de a la de mistral api, a la del puerto desde el que se puede acceder a ollama. No necesitaria una key porque mi puerto esta abierto sino que tendria que apuntar a la aplicacion ollama para que se ejecute. luego podria pedirle un modelo de ollama, en este caso podria usar mistral 7b en lugar de mistral small latest.


1. hay que hacer ollama serve para que se habra la url 
2. luego hay que hacer ollama pull para que descargue el modelo
3. a partir de ahi se pude usar

In [ ]:
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [ ]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [ ]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [ ]:
def build_prompt(query, search_results):
    prompt_template = """
        You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
        Use only the facts from the CONTEXT when answering the QUESTION.

        QUESTION: {question}

        CONTEXT: 
        {context}
        """.strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [ ]:
def llm(prompt):
    response = client.chat.complete(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [ ]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer